**JUAN DAVID VELÁSQUEZ RESTREPO**

In [ ]:
# Importación de paquetes necesarios para la comparación de los clasificadores.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.decomposition import PCA, KernelPCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sb
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# AJUSTE DE LOS PARÁMETROS DEL ESCALIZADOR MEDIANTE EL MÉTODO MIN_MAX A UN
# RANGO DE -1 A 1.
#scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
#  GUARDADO DE LOS DATOS EN UN DATAFRAME DE PANDAS.
datos=pd.read_csv("train.csv",sep=",")
# SE ELIMINAN ALGUNAS COLUMNAS DE POCA RELEVANCIA Y QUE NO SE PUEDEN ASOCIAR
# A UN DATO NUMÉRICO.
datos = datos.drop(['Name','Ticket','Embarked','PassengerId','Fare','Cabin'], axis=1)
# SE ELIMINAN TODAS LAS FILAS CON DATOS NaN.
datos['Age'].fillna(value=datos['Age'].mean(), inplace=True)
datos = datos.dropna()
# SE ASOCIA EL GÉNERO MASCULINO COMO '0' Y EL FEMENINO COMO '1'.
datos['Sex'] = np.where(datos['Sex'] == 'male', 0,1)
# VARIABLE ASOCIADA AL NÚMERO DE CARACTERÍSTICAS DESEADAS.
# GUARDADO DE LAS CARACTERÍSTICAS EN UNA VARIABLE.
X = datos[['Pclass','Sex','SibSp','Parch','Age']]
X = np.array(X)
# GUARDADO DE LAS ETIQUETAS EN UNA VARIABLE.
Y = datos[['Survived']]
Y = np.array(Y)

display(datos)

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,0,22.000000,1,0
1,1,1,1,38.000000,1,0
2,1,3,1,26.000000,0,0
3,1,1,1,35.000000,1,0
4,0,3,0,35.000000,0,0
...,...,...,...,...,...,...
886,0,2,0,27.000000,0,0
887,1,1,1,19.000000,0,0
888,0,3,1,29.699118,1,2
889,1,1,0,26.000000,0,0


**ENTRENAMIENTO DEL ESCALIZADOR Y USO DEL PCA**

In [ ]:
# ADQUISICIÓN DE LAS CARACTERÍSTICAS/ETIQUETAS PARA EL ENTRENAMIENTO Y TESTEO.
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.1, random_state = 101)
# ENTRENAMIENTO DEL ESCALIZADOR CON EL CONJUNTO DE ENTRENAMIENTO.
scaler = scaler.fit(X_train)
# SE CREAN DOS CONJUNTOS DE CARACTERÍSTICAS CON Y SIN REDUCCIÓN DIMENSIONAL.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**REGRESIÓN LOGÍSTICA**

**KNN - K VECINOS MÁS CERCANOS**

**SVM - MÁQUINAS DE SOPORTE VECTORIAL**

**SVM MEJORADA**

In [ ]:
# SELECCIÓN DE HIPERPARÁMETROS DEL CLASIFICADOR. ESTOS SERÁN LOS QUE SE VAN A
# ITERARSE.
parameters = {'kernel':('poly', 'rbf','sigmoid'), 'C':[0.0003, 0.01, 0.03, 0.1, 0.3, 1, 2, 5, 10 ,30,100], 'decision_function_shape': ('ovo', 'ovr'),
              'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}

# SE GUARDA EL TIPO DE CLASIFICADOR EN UNA VARIABLE.
svc = svm.SVC()

# ESTA SECCIÓN HACE LA BÚSQUEDA POR GRADIENTE PARTIENDO DE LOS HIPERPARÁMETROS DADOS.
clf = GridSearchCV(svc, parameters, cv = 5, refit = 'True', verbose = 0)

# ENTRENAMIENTO DEL MODELO CON MEJORES RESULTADOS.
clf.fit(X_train, Y_train)
sorted(clf.cv_results_.keys())

# IMPRESIÓN DE LOS RESULTADOS PARA CADA CLASE Y LAS CARACTERÍSTICAS DEL MODELO
# CON MEJORES HIPERPARÁMETROS.
print("Mejores Hiperparámetros:", clf.best_params_)
print("\n")

'''
FINALMENTE SE REALIZA TODO EL PROCESO DE EVALUACIÓN DEL CLASIFICADOR APLICADO
QUE EN ESTE CASO, ES MÁQUINAS DE SOPORTE VECTORIAL.
IMPLEMENTANDO TÉCNICAS COMO EL ACCURACY, EL MCC Y F1 SCORE.
'''
y_test_predicted = clf.predict(X_test)
y_test_scores = clf.decision_function(X_test)
MCC = matthews_corrcoef(Y_test, y_test_predicted)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_test_predicted)
print("Resultado mediante Accuracy", ACC)
print("\n")

# IMPRESIÓN DE UN DATAFRAME CON LOS RESULTADOS.
report_dict = classification_report(Y_test, y_test_predicted, output_dict=True)
reporte = pd.DataFrame(report_dict)
reporte.drop(['accuracy'], axis=1, inplace = True)
print("Reporte de resultados en cada clase:\n")
display(reporte)
print("\n")

Mejores Hiperparámetros: {'C': 1, 'decision_function_shape': 'ovo', 'gamma': 0.1, 'kernel': 'rbf'}


Resultado mediante MCC: 0.6445327547804199
Resultado mediante Accuracy 0.8222222222222222


Reporte de resultados en cada clase:



,0,1,macro avg,weighted avg
precision,0.786885,0.896552,0.841718,0.834407
recall,0.941176,0.666667,0.803922,0.822222
f1-score,0.857143,0.764706,0.810924,0.817087
support,51.000000,39.000000,90.000000,90.000000


**RED NEURONAL**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import KFold
import tensorflow as tf

num_folds = 5

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = X_train
targets = Y_train

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  model256 = Sequential()
  model256.add(Dense(2048, input_shape = (5,), activation = 'relu', kernel_initializer = 'he_normal'))
  model256.add(Dense(1024, activation = 'relu', kernel_initializer = 'he_normal'))
  model256.add(Dense(512, activation = 'relu', kernel_initializer = 'he_normal'))
  model256.add(Dense(128, activation = 'relu', kernel_initializer = 'he_normal'))
  model256.add(Dense(64, activation = 'relu', kernel_initializer = 'he_normal'))
  model256.add(Dense(1, activation = 'sigmoid'))

  # Checkpoint to save best model
  checkpoint = ModelCheckpoint(f'best_model_{fold_no}.hdf5', monitor = 'accuracy', verbose = 0, save_best_only = True)

  model256.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model256.fit(inputs[train], targets[train],
              batch_size = 16, epochs = 256, callbacks = [checkpoint])

  # Generate generalization metrics
  scores = model256.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model256.metrics_names[0]} of {scores[0]}; {model256.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/256
40/40 [==============================] - 2s 31ms/step - loss: 0.5916 - accuracy: 0.7422
Epoch 2/256
40/40 [==============================] - 1s 30ms/step - loss: 0.4659 - accuracy: 0.8031
Epoch 3/256
40/40 [==============================] - 1s 31ms/step - loss: 0.4925 - accuracy: 0.8047
Epoch 4/256
40/40 [==============================] - 1s 27ms/step - loss: 0.5284 - accuracy: 0.7781
Epoch 5/256
40/40 [==============================] - 1s 28ms/step - loss: 0.4874 - accuracy: 0.8031
Epoch 6/256
40/40 [==============================] - 1s 31ms/step - loss: 0.4412 - accuracy: 0.8141
Epoch 7/256
40/40 [==============================] - 1s 30ms/step - loss: 0.4344 - accuracy: 0.8234
Epoch 8/256
40/40 [==============================] - 1s 29ms/step - loss: 0.4453 - accuracy: 0.8125
Epoch 9/256
40/40 [==============================] - 1s 28ms/step - loss: 0.4277 - accuracy: 0.8172
Epo

In [ ]:
mc = np.array([])
for i in range(1,5):
  model = tf.keras.models.load_model(f'/content/best_model_{i}.hdf5')
  y_pred = np.around(np.array(model.predict(X_test)))
  MCC = matthews_corrcoef(Y_test, y_pred)
  mc = np.append(mc,MCC)

model = tf.keras.models.load_model(f'/content/best_model_{np.argmax(mc)+1}.hdf5')
MCC = matthews_corrcoef(Y_test, y_pred)
print(MCC)

0.6183469424008423


In [ ]:
y_pred = np.around(np.array(model.predict(X_test)))
ACC = accuracy_score(Y_test, y_pred)
print("Resultado mediante Accuracy", ACC)
print("\n")

Resultado mediante Accuracy 0.8111111111111111






> DEDUCCIÓN


Como se observa en el resultado del MCC en el modelo de máquinas de soporte vectorial implementado, es el valor donde obtiene mayor magnitud con respecto a los demás métodos, por tanto, se implementa este modelo para efectuar el modelo de clasificación que será subido a Kaggle.

**PREPARACIÓN DEL ARCHIVO DE CLASIFICACIÓN PARA KAGGLE**

In [ ]:
# PROCESO DE LIMPIEZA Y FILTRADO PREVIO DE LOS DATOS DE VALIDACIÓN.
test = pd.read_csv("test.csv",sep=",")
# PROCESO DE LIMPIEZA Y CLASIFICACION DE LOS DATOS DE VALIDACIÓN.
test['Sex'] = np.where(test['Sex'] == 'male', 0,1)
test = test.drop(['Ticket','Cabin','Embarked','Fare'], axis=1)
test['Age'].fillna(value=datos['Age'].mean(), inplace=True)
test = test.dropna()
passengerId = test['PassengerId'].values
test_X = np.array(test[['Pclass','Sex','SibSp','Parch','Age']].values)
# PROCESO DE ESCALIZACIÓN, APLICACIÓN DEL PCA Y OBTENCIÓN DE LOS RESULTADOS DE
# PREDICCIÓN.
test_X = scaler.transform(test_X)
y_test_predicted = np.around(model.predict(test_X))
display(passengerId.shape)
print(y_test_predicted.shape)

# UNIÓN DE COLUMNAS DEL ID DEL PASAJERO Y SU POSIBLE RESULTADO EN EL TITANIC.
combined = np.vstack((passengerId, np.reshape(y_test_predicted,(418,)))).T
combined = combined.astype(int)
Prediction = pd.DataFrame(combined, columns=['PassengerId', 'Survived'])
Prediction.to_csv('ID_Prediction.csv',index = False)

(418,)

(418, 1)


**CONCLUSIONES**


*   A pesar que la SVM (Máquina de Soporte Vectorial) obtuvo un coeficiente de correlación de Matthews mayor que los demás modelos con un valor de 0.535, en contraste con las otras técnicas, no hubo mayor diferencia pues no hay una variación superior a 0.1, por tanto, las demás técnicas son de igual manera un método viable para el análisis del problema en cuestión.
*   La técnica que obtuvo mayor área bajo la curva a partir del "Receiver operating characteristic" fue la regresión logística, con un valor de 0.827, posteriormente la SVM obtuvo un valor de 0.802, lo cual no implica una diferencia notable entre ambas magnitudes.
*   Es de suma importancia tener presentes los datos los cuales se van a eliminar mediante el método "dropna()" puesto que puede eliminar varias filas completas que podrían ser útiles para la extracción de información del problema, lo cual puede implicar disminución del MCC o de la precisión de los clasificadores a emplear, por tanto, realizar un preprocesamiento de los datos que no se van a utilizar, es primordial para los resultados óptimos de la solución al problema.
*   En el caso de las máquinas de soporte vectorial, al momento de emplear un Kernel de tipo polinomia y a su vez, hacer uso de un grado alto, no implica que el MCC aumente notablemnte puesto que puede sufrir del problema de sobreajuste y al momento de probar el modelo de clasificación con los datos de validación su resultado no es del todo óptimo, dado que fue entrenado específicamente para datos con el comportamiento del conjunto de entrenamiento.







In [ ]:
y_test_predicted = np.around(clf.predict(test_X))
display(passengerId.shape)
print(y_test_predicted.shape)

# UNIÓN DE COLUMNAS DEL ID DEL PASAJERO Y SU POSIBLE RESULTADO EN EL TITANIC.
combined = np.vstack((passengerId, np.reshape(y_test_predicted,(418,)))).T
combined = combined.astype(int)
Prediction = pd.DataFrame(combined, columns=['PassengerId', 'Survived'])
Prediction.to_csv('ID_Prediction.csv',index = False)

(418,)

(418,)
